In [116]:
import pandas as pd
import numpy as np
from  sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

In [117]:
print("Hello World")

Hello World


In [118]:
# Read in the input data
# Data gathered from this link: https://www.kaggle.com/datasets/uciml/sms-spam-collection-dataset/data

df = pd.read_csv('data/spam.csv', encoding='latin1')
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [119]:
# Get rid of columns without any data in them
bad_columns = ['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4']
df.drop(bad_columns, axis=1, inplace=True)
df.columns = ['Category', 'Message']

# Check for missing values
df.isna().sum()

Category    0
Message     0
dtype: int64

In [120]:
# Drop duplicated rows
df = df.drop_duplicates(keep='first')
df.duplicated().sum()

0

In [121]:
# Get the number of each category
num_spam = df[df['Category'] == 'spam'].shape[0]
num_non_spam = df[df['Category'] == 'ham'].shape[0]
print("Spam emails: ", num_spam, f" Percent of total: {100 * num_spam / (num_spam + num_non_spam):.1f}%")
print("Non-spam emails: ", num_non_spam, f" Percent of total: {100 * num_non_spam / (num_spam + num_non_spam):.1f}%")

Spam emails:  653  Percent of total: 12.6%
Non-spam emails:  4516  Percent of total: 87.4%


In [122]:
# Convert the category column to a column of 1's denoting spam, 0's denoting not spam.

mapping = {'ham': 0, 'spam': 1}
df['spam'] = df['Category'].map(mapping)
df = df.drop('Category', axis=1)
df.head()

,Message,spam
0,"Go until jurong point, crazy.. Available only ...",0
1,Ok lar... Joking wif u oni...,0
2,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,U dun say so early hor... U c already then say...,0
4,"Nah I don't think he goes to usf, he lives aro...",0


In [123]:
# Split the data into a training set and test set
x_train, x_test, y_train, y_test = train_test_split(df['Message'], df['spam'], test_size=0.2)

In [124]:
# Feature extraction
vectorizer = TfidfVectorizer()
x_train_tfidf = vectorizer.fit_transform(x_train)
x_test_tfidf = vectorizer.transform(x_test)

x_train_dense = x_train_tfidf.toarray()
x_test_dense = x_test_tfidf.toarray()

In [125]:
# Build the model
import tensorflow as tf
import keras
from keras import layers
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.regularizers import l1_l2

# Using a simpler, statically trained model is fine here. The classification does not have any internal state so using more complex architectures like RNN's is not necessary.

model = keras.Sequential()
# model.add(Dense(64, activation='relu', kernel_regularizer=l1_l2(l1=0.03, l2=0.00)))
# model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(x_train_dense, y_train, epochs=20, batch_size=1, validation_split=0.2)

Epoch 1/20
3308/3308 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - accuracy: 0.9007 - loss: 0.3081 - val_accuracy: 0.9819 - val_loss: 0.0820
Epoch 2/20
3308/3308 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - accuracy: 0.9888 - loss: 0.0489 - val_accuracy: 0.9807 - val_loss: 0.0717
Epoch 3/20
3308/3308 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - accuracy: 0.9939 - loss: 0.0171 - val_accuracy: 0.9807 - val_loss: 0.0753
Epoch 4/20
3308/3308 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - accuracy: 0.9961 - loss: 0.0129 - val_accuracy: 0.9794 - val_loss: 0.0817
Epoch 5/20
3308/3308 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - accuracy: 0.9963 - loss: 0.0082 - val_accuracy: 0.9843 - val_loss: 0.0892
Epoch 6/20
3308/3308 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - accuracy: 0.9986 - loss: 0.0046 - val_accuracy: 0.9855 - val_loss: 0.0968
Epoch 7/20
3308/3308 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - accuracy: 0.9991 - loss: 0.0024 - val_accuracy: 0.9819 - val_loss: 0.1178
Epoch 8/20
3308/3308 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - accuracy: 0.9996 - loss: 0.0016 - 

In [126]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Make predictions on the test data
y_pred_probability = model.predict(x_test_dense)
y_pred = (y_pred_probability > 0.3).astype(int)

# Calculate evaluation metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)

33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 642us/step
Accuracy: 0.9806576402321083
Precision: 0.9523809523809523
Recall: 0.8955223880597015
F1-score: 0.9230769230769231


In [131]:
def classify_email(email_text):
    email_tfidf = vectorizer.transform([email_text])
    prediction = model.predict(email_tfidf)[0]
    print(prediction)
    return "Spam" if prediction >= 0.4 else "Not Spam"

# Example usage
new_email = "Get rich quick! Text GP to this number for more information"
result = classify_email(new_email)
print("Email classification:", result)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
[0.97303116]
Email classification: Spam
